In [48]:
import pandas as pd

#Task 1
Для датафрейма log из материалов занятия создайте столбец source_type по правилам:

-если источник traffic_source равен Yandex или Google, то в source_type ставится organic;

-для источников paid и email из России ставим ad;

-для источников paid и email не из России ставим other;

-все остальные варианты берём из traffic_source без изменений.

In [49]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [64]:
def source(info):
    if ('yandex' in info['traffic_source']) or ('google' in info['traffic_source']):
        return 'organic'
    elif (('paid' in info['traffic_source']) or ('email' in info['traffic_source'])) and ('Russia' in info['region']):
        return 'ad'
    elif (('paid' in info['traffic_source']) or ('email' in info['traffic_source'])) and ('Russia' not in info['region']):
        return 'other'
    else:
        return info['traffic_source']

In [65]:
log['source_type'] = log.apply(source, axis=1)
log

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
...,...,...,...,...,...,...,...
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,other
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,ad
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,ad


#Task 2
В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей.

Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. Выполните действия:

1)Прочитайте содержимое файла с датафрейм.

2)Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.

In [125]:
news_url = pd.read_csv('URLs.txt')
news_url

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


In [126]:
news_url = news_url[news_url.url.str.contains('/[0-9]{8}-', regex=True)]
news_url.head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


#Task 3
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 
Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

In [131]:
rating = pd.read_csv('ratings.csv')
rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [132]:
date = rating[['userId', 'rating']].groupby('userId').count().reset_index()
date = date[date['rating'] > 100]
date['min_date'] = rating[['userId', 'timestamp']].groupby('userId').min().reset_index()['timestamp']
date['max_date'] = rating[['userId', 'timestamp']].groupby('userId').max().reset_index()['timestamp']
date.head()

,userId,rating,min_date,max_date
3,4,204,949778714,949982274
7,8,116,1154389340,1154474527
14,15,1700,997937239,1469330735
16,17,363,1127468587,1127476640
18,19,423,855190091,855195373


In [133]:
date['min_date'] = pd.to_datetime(date['min_date'], unit='s')
date['max_date'] = pd.to_datetime(date['max_date'], unit='s')
date['life'] = date['max_date'] - date['min_date']
date.head()

,userId,rating,min_date,max_date,life
3,4,204,2000-02-05 19:25:14,2000-02-08 03:57:54,2 days 08:32:40
7,8,116,2006-07-31 23:42:20,2006-08-01 23:22:07,0 days 23:39:47
14,15,1700,2001-08-16 04:47:19,2016-07-24 03:25:35,5455 days 22:38:16
16,17,363,2005-09-23 09:43:07,2005-09-23 11:57:20,0 days 02:14:13
18,19,423,1997-02-06 00:48:11,1997-02-06 02:16:13,0 days 01:28:02


In [134]:
result = date['life'].mean()
result

Timedelta('463 days 21:28:27.449612408')

#Task 4
Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе «Материалы для лекции “Продвинутый pandas”» ---- Ноутбуки к лекции «Продвинутый pandas»).
Нужно сформировать две таблицы:

1)таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;

2)аналогичную таблицу по типам выручки с указанием адреса клиента.

In [170]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

In [171]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

In [172]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

In [173]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [165]:
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [177]:
a = rzd.merge(auto, on='client_id', how='outer').fillna(0)
a

,client_id,rzd_revenue,auto_revenue
0,111,1093.0,0.0
1,112,2810.0,0.0
2,113,10283.0,57483.0
3,114,5774.0,83.0
4,115,981.0,912.0
5,116,0.0,4834.0
6,117,0.0,98.0


In [178]:
b = a.merge(air, on='client_id', how='outer').fillna(0)
b

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [179]:
c = b.merge(client_base, on='client_id')
c

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4
